## The third Lab-assignment (02/10/2022, 50 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points). Fomulate your domain problem: Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
'''
Please write you answer here:

Research question:
I have chosen about the Internation space station which is moving 28000 km/h, so finding how the location changes very fast and currently where it is?
Its a simple API file, which is having ISS current loaction and it retrive the current latitude and longitude of the ISS along with a unix timestamp for valid location and time.
This API doesn't contain any inputs.

Question 2 (10 points). Collect your data to answer the research problem: Write python code to collect 1000 data samples you discussed above.

In [14]:
import re
import pandas

# the ISS api shown below has data about ISS current location
iss_api= 'http://api.open-notify.org/iss-now.json'

iss_res=requests.get(iss_api)

df=pandas.read_json(iss_res.text)

print(df)


                    timestamp  iss_position  message
latitude  2022-02-28 11:59:00       -9.5412  success
longitude 2022-02-28 11:59:00        4.4441  success


Question 3 (10 points). Understand the data quality: Search a second hand dataset (any dataset) from kaggle or other websites. Describe the data quality problem of the dataset and explain your strtegy to clean the data.

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):
'''


Please write you answer here:

-> I considered a world university ranking data from Kaggle.
Dataset:
. Link: https://www.kaggle.com/padhmam/qs-world-university-rankings-2017-2022
. There are 14 data columns in the dataset.
. The target column is having many countires in the wolrd.

Steps to Clean the University rankings data:
. we convert all the character data into lowercase.
. Stopwords will be removed from the data.
. Remove any special character or unwanted space in the data.
. Remove null values from the data
. Perform stemming and Lemmatization on the data.
. Check if there is any irrelevent data.
. If there is any irrelevent, we remove the data and we choose the data based on requirement country and by sorting the ranking column,
'''

Question 4 (20 points). Data cleaning: There are two datasets TwADR-L (from Twitter) and AskAPatient (Link: https://zenodo.org/record/55013#.YgU2NN-ZO4T) for medical concept
normalization. However, the two datasets have serious data quality problems. Please read the introduction of the datasets and clean the two datasets by following the steps in the statement.

In the original dataset, the TwADR-L had 48,057 training, 1,256 validation and 1,427 test examples. The test set (all
test samples from 10 folds combined) consists of 765 unique phrases and 273 unique classes (medical concepts). The AskAPatient dataset contained 156,652 training, 7,926 validation, and 8,662 test examples. The entire test set (all test samples
from 10 folds combined) consists of 3,749 unique phrases and 1,035 unique classes (medical concepts). The authors
randomly split each dataset into ten equal folds, ran 10-fold cross validation and reported the accuracy averaged across the
ten folds. 

We found that, in the original data set, many phrase-label pairs appeared multiple times within the same training data file
and also across the training and test data sets in the same fold. In the AskAPatient data set, on average 35.82% of the test data overlapped with training data in the same fold. In the Twitter (TwADR-L) dataset, on average 8.62% of the test set had an overlap with the training data in the same fold. Having a large overlap between the training and the test data can potentially
introduce bias in the model and contribute to high accuracy. It is not unlikely that the high model performance reported in the original paper may be triggered by the the large overlap between the training and test sets.

Therefore to remove the bias, we further cleaned and recreated the training, validation, and test sets such that each
phrase-label pair appears only once in the entire dataset (either in training, validation or test set).

(1) First, we combined all examples in training, validation and test data from the original data set and then removed all
duplicate phrase-label pairs (examples that have the same phrase and label pair and appear more than once in training/validation/test datasets). Table II shows the statistics of the new dataset (after removing duplicates). The Twitter data set had 3,157 unique phrase-label pairs and 2,220 unique labels (medical concepts) while 173 phrases had multiple labels (i.e., they were assigned to more than one label). Many concepts had only one example, and the concept that had the most number
of examples had 36 phrases. On average, each concept had 1.42 examples. The AskAPatient data set had 4,496 unique phrase-label pairs, 1,036 unique labels while 26 phrases had multiple labels. Table III shows examples of phrases that had multiple labels. For example, ‘mad’ can be mapped to ‘anger’ or ‘rage’ and ‘sore’ can be mapped to ‘pain’ or ‘myalgia’.

(2) Second, we remove all concepts that had less than five examples. The statistics of the final data are shown in Table IV.

(3) Third, we divide all examples without multiple labels into random 10 folds such that each unique phrase-label pair
appears once in one of the 10 test sets. We add the pairs with multiple labels into the training data. This final 10-folds
dataset is used in all our experiments.

(The original paper can be download on canvas)

In [1]:
import warnings
import pandas as pd
import glob
import os
dtst=['TwADR-L','AskAPatient']
resdf=pd.DataFrame()
path='C:/Users/vishn/Downloads/datasets/datasets'
#print(path)
for name in dtst:
    rslt={}
    ldf=pd.DataFrame()
    file=path+'/'+name
    #print(name,file)
    os.chdir(file)
    os.chmod(file,0o777)
    for files in glob.glob('*.txt'):
        #print (files)
        rd=pd.read_csv(files,sep='\t',header = None, encoding= 'unicode_escape')
        ldf=ldf.append(rd)
    ldf.reset_index(drop=True)
    ldf.columns=['ID','label','Phrase']
    ldf['label']=ldf['label'].str.upper()
    ldf['Phrase']=ldf['Phrase'].str.upper()
    ldf.drop_duplicates(subset=['label','Phrase'],inplace=True)
    ldf["Phrase-label"]=ldf['Phrase']+ldf['label']
    rslt['#Unique phrases'] =ldf['label'].nunique()
    rslt['#Unique labels'] =len(ldf['Phrase'].unique())
    rslt['# unique phrase-label pairs'] =ldf["Phrase-label"].nunique()
    ldf1 = pd.DataFrame(ldf['Phrase'].value_counts())
    rslt["# phrases with multiple labels"]=ldf1[ldf1['Phrase'] > 1].shape[0]
    rslt['Min # examples per label'] = ldf['label'].value_counts().values.min()
    rslt['Max # examples per label'] = ldf['label'].value_counts().values.max()
    rslt['Avg # examples per label'] = round(ldf['label'].value_counts().mean(), 2)
    resdf = resdf.append(rslt, ignore_index=True)
resdf = resdf.T
resdf = resdf.astype({0: int, 1: int})
resdf.columns = ['TwADR-L', 'AskAPatient']
resdf


,TwADR-L,AskAPatient
#Unique phrases,2220,1038
#Unique labels,2944,4470
# unique phrase-label pairs,3157,4507
# phrases with multiple labels,173,35
Min # examples per label,1,1
Max # examples per label,36,141
Avg # examples per label,1,4


In [4]:
res1_df=pd.DataFrame()
for name in dtst:
    res1={}
    index_list = []
    for i in range(ldf.shape[0]):
        if ldf['label'].value_counts()[ldf.iloc[i]['label']] < 5:
            index_list.append(i)
    ldf.drop(ldf.index[index_list],inplace=True)
    res1['#Unique phrases'] =ldf['label'].nunique()
    res1['#Unique labels'] =len(ldf['Phrase'].unique())
    res1['# unique phrase-label pairs'] =ldf["Phrase-label"].nunique()
    #occur = df.groupby(['label']).size()
    #print(occur[1]>1)#['label']>1)
    ldf1 = pd.DataFrame(ldf['Phrase'].value_counts())
    res1["# phrases with multiple labels"]=ldf1[ldf1['Phrase'] > 1].shape[0]
    res1['Min # examples per label'] = ldf['label'].value_counts().values.min()
    res1['Max # examples per label'] = ldf['label'].value_counts().values.max()
    res1['Avg # examples per label'] = round(ldf['label'].value_counts().mean(), 2)
    res1_df = res1_df.append(res1, ignore_index=True)
res1_df = res1_df.T
res1_df = res1_df.astype({0: int, 1: int})
res1_df.columns = ['TwADR-L', 'AskAPatient']
res1_df


,TwADR-L,AskAPatient
#Unique phrases,233,221
#Unique labels,2563,2518
# unique phrase-label pairs,2583,2537
# phrases with multiple labels,19,18
Min # examples per label,3,5
Max # examples per label,135,135
Avg # examples per label,11,11
